In [3]:
import imp
import numpy as np
import pandas as pd
import xgboost as xgb
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import StandardScaler

In [19]:
problem = imp.load_source('', 'problem.py')
X_df, y_array = problem.get_train_data()

In [20]:
X_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [14]:
X_df['log_PAX'] = y_array
cols = X_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
X_df = X_df[cols]
X_df.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159


In [7]:
mean_encode_dep = X_df.groupby('Departure')['log_PAX'].mean()
mean_encode_ar = X_df.groupby('Arrival')['log_PAX'].mean()
X_df.loc[:, 'dep_encod'] = X_df['Departure'].map(mean_encode_dep)
X_df.loc[:, 'ar_encod'] = X_df['Arrival'].map(mean_encode_ar)
X_df.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,dep_encod,ar_encod
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647,11.272394,11.082126
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734,10.760347,10.694978
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883,10.682832,11.434573
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202,10.998076,11.257095
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159,10.682832,11.226356


In [21]:
X_encoded = X_df
X_encoded.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [52]:
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix = 'd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded.loc[:, 'd_ATL': 'd_SFO'] = X_encoded.loc[:, 'd_ATL': 'd_SFO'].mul(
    X_encoded.loc[:, 'dep_encod'], axis='rows')
X_encoded.loc[:, 'a_ATL': 'a_SFO'] = X_encoded.loc[:, 'a_ATL': 'a_SFO'].mul(
    X_encoded.loc[:, 'ar_encod'], axis='rows')

X_encoded.head()

,log_PAX,DateOfDeparture,WeeksToDeparture,std_wtd,dep_encod,ar_encod,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO
0,12.331296,2012-06-19,12.875000,9.812647,11.272394,11.082126,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,11.272394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,11.082126,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,10.775182,2012-09-10,14.285714,9.466734,10.760347,10.694978,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,10.760347,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.694978,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,11.083177,2012-10-05,10.863636,9.035883,10.682832,11.434573,0.000000,0.0,0.0,10.682832,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,11.434573,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,11.169268,2011-10-09,11.480000,7.990202,10.998076,11.257095,10.998076,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,11.257095,0.0,0.0,0.0,0.000000
4,11.269364,2012-02-21,11.450000,9.517159,10.682832,11.226356,0.000000,0.0,0.0,10.682832,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,11.226356


In [ ]:
X_empty = pd.DataFrame(columns=X_encoded.columns)
X_encoded ['test'] = X_encoded['std_wtd']*X_encoded['WeeksToDeparture']

X_encoded.loc[:, 'WeeksToDeparture' : 'test'] = StandardScaler().fit_transform(
    X_encoded.loc[:, 'WeeksToDeparture' : 'test'])
#X_encoded.loc[:,'test'] = StandardScaler().fit_transform(X_encoded.loc[:,'test'])
X_encoded.head()

In [16]:
external_data = pd.read_csv('submissions/starting_kit/external_data.csv')
external_data = external_data[['Date', 'AirPort', 'Mean TemperatureC', 'MeanDew PointC', 
                               'Mean Humidity', 'Mean VisibilityKm', 'Mean Wind SpeedKm/h', 
                               'CloudCover']]
external_data = external_data.rename(
            columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival'],
            right_on=['DateOfDeparture', 'Arrival'],
            sort=False)
X_encoded.head()

KeyError: "['AirPort', 'Date'] not in index"

In [22]:
external_data = pd.read_csv('submissions/starting_kit/external_data.csv')
external_data = external_data[['DateOfDeparture', 'Departure', 'Arrival', 'dep_encod', 'ar_encod', 'Distance',
                                'Mean TemperatureC', 'MeanDew PointC', 'Mean Humidity', 'Min VisibilitykM',
                                'Max Wind SpeedKm/h', 'Precipitationmm','CloudCover','Events']]
        
X_encoded = pd.merge(X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Departure', 'Arrival'],
            right_on=['DateOfDeparture','Departure', 'Arrival'],
            sort=False)
X_encoded.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,dep_encod,ar_encod,Distance,Mean TemperatureC,MeanDew PointC,Mean Humidity,Min VisibilitykM,Max Wind SpeedKm/h,Precipitationmm,CloudCover,Events
0,2012-06-19,ORD,DFW,12.875000,9.812647,11.272394,11.082126,1292,29,21,63,16,48,0.00,5,NaN
1,2012-09-10,LAS,DEN,14.285714,9.466734,10.760347,10.694978,1010,25,-6,14,16,35,0.00,3,NaN
2,2012-10-05,DEN,LAX,10.863636,9.035883,10.682832,11.434573,1387,19,16,77,8,24,0.00,5,Fog
3,2011-10-09,ATL,ORD,11.480000,7.990202,10.998076,11.257095,977,19,10,58,16,23,0.00,1,NaN
4,2012-02-21,DEN,SFO,11.450000,9.517159,10.682832,11.226356,1554,12,8,79,3,24,0.00,7,NaN


In [23]:
X_encoded.isnull().sum()

DateOfDeparture          0
Departure                0
Arrival                  0
WeeksToDeparture         0
std_wtd                  0
dep_encod                0
ar_encod                 0
Distance                 0
Mean TemperatureC        0
MeanDew PointC           0
Mean Humidity            0
Min VisibilitykM         0
Max Wind SpeedKm/h       0
Precipitationmm          0
CloudCover               0
Events                5490
dtype: int64

In [24]:
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

#mean encoding
X_encoded.loc[:, 'd_ATL': 'd_SFO'] = X_encoded.loc[:, 'd_ATL': 'd_SFO'].mul(X_encoded.loc[:, 'dep_encod'], axis='rows')
X_encoded.loc[:, 'a_ATL': 'a_SFO'] = X_encoded.loc[:, 'a_ATL': 'a_SFO'].mul(X_encoded.loc[:, 'ar_encod'], axis='rows')
X_encoded.head()

,DateOfDeparture,WeeksToDeparture,std_wtd,dep_encod,ar_encod,Distance,Mean TemperatureC,MeanDew PointC,Mean Humidity,Min VisibilitykM,Max Wind SpeedKm/h,Precipitationmm,CloudCover,Events,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO
0,2012-06-19,12.875000,9.812647,11.272394,11.082126,1292,29,21,63,16,48,0.00,5,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,11.272394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,11.082126,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,2012-09-10,14.285714,9.466734,10.760347,10.694978,1010,25,-6,14,16,35,0.00,3,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,10.760347,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.694978,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,2012-10-05,10.863636,9.035883,10.682832,11.434573,1387,19,16,77,8,24,0.00,5,Fog,0.000000,0.0,0.0,10.682832,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,11.434573,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,2011-10-09,11.480000,7.990202,10.998076,11.257095,977,19,10,58,16,23,0.00,1,NaN,10.998076,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,11.257095,0.0,0.0,0.0,0.000000
4,2012-02-21,11.450000,9.517159,10.682832,11.226356,1554,12,8,79,3,24,0.00,7,NaN,0.000000,0.0,0.0,10.682832,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,11.226356


In [ ]:


X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
#X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
#X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
#X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

#X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
#X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))

#X_encoded = X_encoded.drop(['year', 'month', 'weekday', 'n_days'], axis=1)
X_encoded = X_encoded.drop(['weekday'], axis=1)

X_encoded = X_encoded.drop('DateOfDeparture', axis=1)

X_encoded.head()

In [ ]:

c
X_encoded.head()


In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)

        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))

        X_encoded = X_encoded.drop(['year', 'month', 'day', 'weekday', 'week', 'n_days'], axis=1)

        X_encoded.loc[:, 'WeeksToDeparture' : 'std_wtd'] = StandardScaler().fit_
                    transform(X_encoded.loc[:, 'WeeksToDeparture' : 'std_wtd'])

        X_encoded = X_encoded.drop(['DateOfDeparture', 'std_wtd'], axis=1)
        X_array = X_encoded.values
        return X_array

In [ ]:
import pandas as pd
import os


def _encode(X_df):
    X_encoded = X_df
    #path = os.path.dirname(__file__)
    #data_weather = pd.read_csv(os.path.join(path, 'external_data.csv'))
    #X_weather = data_weather[['Date', 'AirPort', 'Events']]
    #X_weather = X_weather.rename(
        #columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
    #X_encoded = pd.merge(
        #X_encoded, X_weather, how='left',
        #left_on=['DateOfDeparture', 'Arrival'],
        #right_on=['DateOfDeparture', 'Arrival'],
        #sort=False)

    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
    X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
    X_encoded = X_encoded.drop([]'Departure', 'Arrival'], axis=1)

    
    X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
    
    return X_encoded


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        X_encoded = _encode(X_df)
        self.columns = X_encoded.columns
        return self

    def transform(self, X_df):
        X_encoded = _encode(X_df)
        X_empty = pd.DataFrame(columns=self.columns)
        X_encoded = pd.concat([X_empty, X_encoded], axis=0, sort=False)
        X_encoded = X_encoded.fillna(0)

        # Reorder/Pick columns from train
        X_encoded = X_encoded[list(self.columns)]
        # Check that columns of test set are the same than train set
        assert list(X_encoded.columns) == list(self.columns)
        X_array = X_encoded.values
        return X_array